In [ ]:
import sys
!{sys.executable} -m pip install pillow numpy pandas torch matplotlib tqdm sklearn ipywidgets seaborn fastai torchsummary
!pip install -U albumentations

In [ ]:
# !pip install -U torchvision

In [ ]:
import torch
import os
import torchvision
import pandas as pd
import PIL
import torchsummary

from fastai.vision import *
from fastai.vision.all import *

# os.environ['TORCH_HOME'] = '~/pytorch/torch_home'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# torch.backends.cudnn.enabled = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
root = Path('../input/sorghum-id-fgvc-9')
# root = Path('../input/sorghumfgvc9png512512/sorghum-fgvc9-png-512')

In [ ]:
train_df = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv').dropna()
train_df

# Outliers on train set

In [ ]:
outliers = ['29-33-477', '29-34-965', '29-36-468', '29-43-957', '29-45-460', '29-46-961', '29-48-469', '29-49-960', '29-51-465', '30-06-475', '30-07-971', '30-09-467', '30-59-251', '31-00-751', '31-02-238', '31-17-229', '31-18-730', '31-20-230', '31-21-751', '31-23-234', '31-24-729', '31-30-733', '31-32-233', '31-33-750', '31-35-254']
outliers = list(map(lambda id: f'2017-06-11__13-{id}.png', outliers))

In [ ]:
PIL.Image.open(root/f'train_images/{outliers[0]}').resize((128, 128))

Remove these outliers

In [ ]:
train_df.drop(train_df[train_df['image'].isin(outliers)].index, inplace=True)
train_df = train_df.dropna().reset_index(drop=True)

In [ ]:
actual_images = [img.name for img in (root/'train_images').ls() if img.name in train_df.image.to_list()]
train_df = train_df[train_df.image.isin(actual_images)]
train_df

# Outliers on Test set

There's nothing we can do. just know what there is.

In [ ]:
PIL.Image.open(root/f'test/212116519.png').resize((128, 128))

# Transforms

In [ ]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
import albumentations as A
import torchvision.transforms as T

def get_train_aug(sz):
    csz = max(8, sz // 50)
    return A.Compose([
#         A.CenterCrop(p=1.0, width=896, height=896),
#         A.Resize(p=1.0, width=rsz, height=rsz),
#         A.RandomCrop(p=1.0, width=sz, height=sz),
        A.RandomResizedCrop(width=sz, height=sz, scale=(0.5, 1.0)),
        A.Flip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(),
        A.HueSaturationValue(),
        A.OneOf([
            A.RandomBrightnessContrast(p=0.5),
            A.RandomGamma(p=0.5),
        ], p=0.5),
        A.OneOf([
            A.GaussNoise(p=0.3),
            A.ISONoise(p=0.3),
        ], p=0.25),
        A.OneOf([
            A.GridDropout(ratio=0.33, p=0.1),
            A.CoarseDropout(max_holes=48, min_holes=8, max_height=2*csz, max_width=2*csz, min_height=csz, min_width=csz, p=0.2)
        ], p=0.5),
    ])

def get_valid_aug(size):
    return A.Compose([
        A.Resize(p=1.0, width=size, height=size),
    ])

In [ ]:
from fastai.vision.all import *
from fastai import *

item_tfms = AlbumentationsTransform(get_train_aug(896), get_valid_aug(896))

dls = ImageDataLoaders.from_df(train_df, root/'train_images',
                               valid_pct=0.3332,
                               fn_col=0, label_col=1,
                               num_workers=48, bs=4,
                               item_tfms=item_tfms,
                               batch_tfms=[Normalize.from_stats(*imagenet_stats)])

In [ ]:
dls.show_batch()

# Select Model

In [ ]:
model = torchvision.models.densenet201
# model = torchvision.models.resnet34
# model = torchvision.models.efficientnet_b4

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BCNN(nn.Module):
    
    def __init__(
        self,
        arch=models.resnet50,
        cut=-2,
        pretrained=True,
        fine_tune=False,
        dropout=0.25,
        n_outputs=100,
        feature_map_size=7,
    ):
        super().__init__()
        
        model = arch(pretrained=pretrained)
        
        self.feature_map_size = feature_map_size
        self.out_features = list(model.children())[-1].in_features
        self.n_outputs = n_outputs
        
        # freezing parameters
        if not fine_tune:
            for param in model.parameters():
                param.requires_grad = False
        else:
            for param in resnet.parameters():
                param.requires_grad = True

        layers = list(model.children())[:cut]
        self.features = nn.Sequential(*layers) #.cuda()
        
        self.fc = nn.Linear(self.out_features ** 2, self.n_outputs)
        self.dropout = nn.Dropout(dropout)
        
        # Initialize the fc layers.
        nn.init.xavier_normal_(self.fc.weight.data)
        
        if self.fc.bias is not None:
            torch.nn.init.constant_(self.fc.bias.data, val=0)
        
    def forward(self, x):
        ## X.shape = bs, 3, sz, sz
        ## N = bs; batch size
        N = x.size()[0]
        
        ## x.shape = bs, output channels of arch, sz', sz'
        x = self.features(x)
        x = F.relu(x, inplace=True)
        
        # Classical bilinear pooling
        x = x.view(N, self.out_features, self.feature_map_size ** 2)
        x = self.dropout(x)
        
        # Batch matrix multiplication
        x = torch.bmm(x, torch.transpose(x, 1, 2)) / (self.feature_map_size ** 2) 
        x = x.view(N, self.out_features ** 2)
        
        # Normalization
        x = torch.sqrt(x + 1e-5)
        x = F.normalize(x)
        
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

In [ ]:
# # if use this, update and learn with not "vision_learner" to "Learner"
# model = BCNN(arch=models.densenet161, cut=-1, n_outputs=100, dropout=0.12).to(device)
# model = BCNN(arch=models.resnet50, cut=-2, n_outputs=100).to(device)

## Load model (including weights if exists previous)

In [ ]:
###### from fastai.metrics import error_rate, accuracy

model_name = model.__name__ + '_V26'
save_dir = Path('.')
save_path = save_dir/model_name
load_path = Path('../input/sorghum-cultivar-100-2') #/model_name

learn = vision_learner(dls, model, metrics=accuracy, path='.', model_dir='.', loss_func=LabelSmoothingCrossEntropy(0.2))

print('Model name:', model_name)
print('Model load from:', load_path/f'{model_name}.pth')
print('exists? ', (load_path/f'{model_name}.pth').exists())
print('Model save to:', save_path)

if (load_path/f'{model_name}.pth').exists():
    print('Successfully load model from:', load_path/f'{model_name}.pth')
    learn.load(load_path/model_name)

learn.save(model_name)

In [ ]:
learn.summary()

# Fine tune

In [ ]:
EPOCHS = 8
print(f'fine_tune(epoch={EPOCHS})')
learn.fine_tune(EPOCHS, cbs=[
    ShowGraphCallback(),
    TerminateOnNaNCallback(),
    SaveModelCallback(monitor='accuracy', min_delta=0.01/100, fname=model_name), # +0.01% accuracy
    EarlyStoppingCallback(patience=5),
])

# Save model

In [ ]:
learn.save(model_name)

# Predict

In [ ]:
learn.predict(root/'test/1000005362.png')

In [ ]:
learn.show_results(max_n=12)

In [ ]:
test_images = get_image_files(root/'test')
test_dataloader = learn.dls.test_dl(test_images)
preds, _ = learn.get_preds(dl=test_dataloader)
class_idxs = [pred.argmax(dim=0) for pred in preds]

In [ ]:
results = [dls.vocab[i] for i in class_idxs]
len(test_images), len(results)

# Submission

In [ ]:
image_names = list(map(lambda s: str(s).split('/')[-1], test_images))
submissions = pd.DataFrame(list(zip(image_names, results)), columns=['filename', 'cultivar']).sort_values(by='filename')
submissions.head()

In [ ]:
submissions.to_csv('submission.csv', index=False)